In [1]:
import pandas as pd

In [ ]:
def carregar_df(caminho_arquivo, colunas_alvo):
    return pd.read_csv(caminho_arquivo, usecols=colunas_alvo, sep=';')

def processar_df(ano, caminho_arquivo, colunas_alvo, colunas_finais):
    df_ano = carregar_df(caminho_arquivo, colunas_alvo)
    df_ano.columns = colunas_finais
    return df_ano  

In [2]:
colunas_finais = ['DATA',
'HORA',
'NATUREZA_ACIDENTE',
'BAIRRO',
'BAIRRO_CRUZAMENTO',
'TIPO',
'AUTOMOVEL_ENVOLVIDO',
'MOTO_ENVOLVIDA',
'CICLO_MOTOR_ENVOLVIDO',
'CICLISTA_ENVOLVIDO',
'PEDESTRE_ENVOLVIDO',
'ONIBUS_ENVOLVIDO',
'CAMINHAO_ENVOLVIDO',
'VIATURA_ENVOLVIDA',
'OUTROS_ENVOLVIDO',
'NUM_VITIMAS',
'NUM_VITIMAS_FATAIS']
mapeamento_2019 = ['DATA',
'hora',
'natureza_acidente',
'bairro',
'bairro_cruzamento',
'tipo',
'auto',
'moto',
'ciclom',
'ciclista',
'pedestre',
'onibus',
'caminhao',
'viatura',
'outros',
'vitimas',
'vitimasfatais']
mapeamento_2020_2021 = ['data',
'hora',
'natureza_acidente',
'bairro',
'bairro_cruzamento',
'tipo',
'auto',
'moto',
'ciclom',
'ciclista',
'pedestre',
'onibus',
'caminhao',
'viatura',
'outros',
'vitimas',
'vitimasfatais']
mapeamento_2022 = ['data',
'hora',
'natureza',
'bairro',
'bairro_cruzamento',
'tipo',
'auto',
'moto',
'ciclom',
'ciclista',
'pedestre',
'onibus',
'caminhao',
'viatura',
'outros',
'vitimas',
'vitimasfatais']

# Processamento das bases de acidentes 2019-2022

## Carregamento das bases e adequação das colunas

In [4]:
pre_processamento = {
    2019:{ 'caminho_arquivo':'data/raw/acidentes-2019.csv', 'colunas_alvo':mapeamento_2019},
    2020:{ 'caminho_arquivo':'data/raw/acidentes_2020-novo.csv', 'colunas_alvo':mapeamento_2020_2021},
    2021:{ 'caminho_arquivo':'data/raw/acidentes2021.csv', 'colunas_alvo':mapeamento_2020_2021},
    2022:{ 'caminho_arquivo':'data/raw/acidentes2022.csv', 'colunas_alvo':mapeamento_2022}   
}

In [5]:
pos_processamento = []
for ano in range(2019,2023,1):
    pos_processamento.append(processar_df(ano, pre_processamento[ano]['caminho_arquivo'], 
                                          pre_processamento[ano]['colunas_alvo'], 
                                          colunas_finais))
    

In [6]:
base_bruta = pd.concat(pos_processamento)


## Otimização da base

In [8]:
colunas_a_zerar = ['AUTOMOVEL_ENVOLVIDO', 'MOTO_ENVOLVIDA',
       'CICLO_MOTOR_ENVOLVIDO', 'CICLISTA_ENVOLVIDO', 'PEDESTRE_ENVOLVIDO',
       'ONIBUS_ENVOLVIDO', 'CAMINHAO_ENVOLVIDO', 'VIATURA_ENVOLVIDA',
       'OUTROS_ENVOLVIDO', 'NUM_VITIMAS_FATAIS']

Valores ausentes durante a descrição dos veiculos informados serão definidos como 0 pois não foram totalmente preenchidos ou constam-se como ausentes durante o acidente.

In [9]:
base_bruta[colunas_a_zerar] = base_bruta[colunas_a_zerar].fillna(0)

In [10]:
base_removida = base_bruta.dropna()

### Processamento dos campos  data

In [60]:
def gerar_coluna_data(df, coluna_data):
    if coluna_data not in df.columns:
        print(f"A coluna '{coluna_data}' não existe no DataFrame.")
        return df
    
    df[coluna_data] = pd.to_datetime(df[coluna_data])
    df['DIA'] = df[coluna_data].dt.day
    df['MES'] = df[coluna_data].dt.month
    df['ANO'] = df[coluna_data].dt.year
    return df

In [ ]:
base_otimizada = gerar_coluna_data(base_removida, "DATA")

### Processamento do campo hora

In [83]:
def classificar_turnos(df, nome_coluna_hora):
    if nome_coluna_hora not in df.columns:
        print(f"A coluna '{nome_coluna_hora}' não existe no DataFrame.")
        return df
    
    # Remove registros com formato de hora inválido
    df = df[df[nome_coluna_hora].apply(verificar_hora_valida)]
    
    df[nome_coluna_hora] = pd.to_datetime(df[nome_coluna_hora])
    df['TURNO'] = df[nome_coluna_hora].apply(lambda x: classificar_turno(x.time() if isinstance(x, pd.Timestamp) else None))

    return df

def verificar_hora_valida(hora):
    try:
        pd.to_datetime(hora)
        return True
    except ValueError:
        return False

def classificar_turno(hora):
    if hora is None:
        return None

    hora_str = hora.strftime('%H:%M:%S')
    
    if '06:00:00' <= hora_str < '12:00:00':
        return 'MANHÃ'
    elif '12:00:00' <= hora_str < '18:00:00':
        return 'TARDE'
    elif '18:00:00' <= hora_str <= '23:59:59':
        return 'NOITE'
    else:
        return 'MADRUGADA'

In [ ]:
base_otimizada = classificar_turnos(base_otimizada,'HORA')

O ano de 2022 tinha sido considerado até a confirmação que a base nesse ano não disponibilizaria o campo HORA, sendo assim inviável para a utilização nessa etapa, com isso o escopo foi reduzido para 2019-2021. Com isso, assim que a base seja completada por sua fonte, ela possa integrar a base final sem qualquer nescessidade de revisão.

## Exportação da base

In [88]:
base_otimizada.to_parquet('base_acidentes_processada')